In [5]:
import numpy as np
import tensorflow 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU, Conv2D, Conv1D, MaxPooling2D,Dropout,UpSampling2D,UpSampling3D,concatenate,Conv2DTranspose,Add
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten


In [7]:
inputs = tensorflow.keras.Input(shape=( 256,256, 1))
conv_layer = Conv2D(16, (3,3), activation = LeakyReLU(alpha=0.01), padding = 'same')(inputs)
conv_layer1 = Conv2D(16, (3,3), activation = LeakyReLU(alpha=0.01), padding = 'same')(conv_layer)
pool1 =  Dropout(0.3)(conv_layer1)
conv_layer1 = Conv2D(16, (3,3), activation = LeakyReLU(alpha=0.01), padding = 'same')(pool1)
conv_layer1 = Add()([conv_layer, conv_layer1])

conv_layer_1s = Conv2D(32, (3,3), activation = LeakyReLU(alpha=0.01), padding = 'same', strides=(2,2))(conv_layer1)

conv_layer2 = Conv2D(32,(3,3) , activation = LeakyReLU(alpha=0.01), padding = 'same')(conv_layer_1s)
pool2 =  Dropout(0.3)(conv_layer2)
conv_layer2 = Conv2D(32, (3,3), activation = LeakyReLU(alpha=0.01), padding = 'same')(pool2)
conv_layer2 = Add()([conv_layer_1s, conv_layer2])

conv_layer_2s = Conv2D(64, (3,3), activation = LeakyReLU(alpha=0.01), padding = 'same', strides=(2, 2))(conv_layer2)

conv_layer3 = Conv2D(64,(3,3) , activation = LeakyReLU(alpha=0.01), padding = 'same')(conv_layer_2s)
pool3 =  Dropout(0.3)(conv_layer3)
conv_layer3 = Conv2D(64, (3,3),activation = LeakyReLU(alpha=0.01), padding = 'same')(pool3)
conv_layer3 = Add()([conv_layer_2s, conv_layer3])

conv_layer_3s = Conv2D(128, (3, 3), activation = LeakyReLU(alpha=0.01), padding = 'same', strides=(2, 2))(conv_layer3)

conv_layer4 = Conv2D(128,(3,3) , activation = LeakyReLU(alpha=0.01), padding = 'same')(conv_layer_3s)
pool4 =  Dropout(0.3)(conv_layer4)
conv_layer4 = Conv2D(128, (3,3), activation = LeakyReLU(alpha=0.01), padding = 'same')(pool4)
conv_layer4 = Add()([conv_layer_3s, conv_layer4])


conv_layer_4s = Conv2D(256, (3, 3), activation = LeakyReLU(alpha=0.01), padding = 'same', strides=(2,2))(conv_layer4)

conv_layer5 = Conv2D(256,(3,3) , activation = LeakyReLU(alpha=0.01), padding = 'same')(conv_layer_4s)
pool5 =  Dropout(0.3)(conv_layer5)
conv_layer5 = Conv2D(256, (3,3), activation = LeakyReLU(alpha=0.01), padding = 'same')(pool5)
conv_layer5 = Add()([conv_layer_4s, conv_layer5])

layer_6 = UpSampling2D()(conv_layer5)

con1 = concatenate([layer_6, conv_layer4])

up1 = Conv2D(128,(3,3) , activation = LeakyReLU(alpha=0.01), padding = 'same')(con1)
up1 = Conv2D(128, (1,1),activation = LeakyReLU(alpha=0.01), padding = 'same')(up1)

layer_7 = UpSampling2D()(up1)

con2 = concatenate([layer_7, conv_layer3])

up2 = Conv2D(64,(3,3) , activation = LeakyReLU(alpha=0.01), padding = 'same')(con2)
up2 = Conv2D(64, (1,1),activation = LeakyReLU(alpha=0.01), padding = 'same')(up2)

seg1 = Conv2D(4,(3,3) , activation = LeakyReLU(alpha=0.01), padding = 'same')(up2)
seg1 = UpSampling2D()(seg1)

layer_8 = UpSampling2D()(up2)

con3 = concatenate([layer_8, conv_layer2])

up3 = Conv2D(32,(3,3) ,activation = LeakyReLU(alpha=0.01), padding = 'same')(con3)
up3 = Conv2D(32, (1,1), activation = LeakyReLU(alpha=0.01), padding = 'same')(up3)

seg2 = Conv2D(4,(3,3) , activation = LeakyReLU(alpha=0.01), padding = 'same')(up3)

layer_9 = UpSampling2D()(up3)

con4 = concatenate([layer_9, conv_layer1])

layer_10 = Conv2D(32, (1,1), activation = LeakyReLU(alpha=0.01), padding = 'same')(con4)

seg3 = Conv2D(4,(3,3) , activation = LeakyReLU(alpha=0.01), padding = 'same')(layer_10)

added_seg12 =Add()([seg1, seg2])
added_seg12= UpSampling2D()(added_seg12)
added_seg123= Add()([added_seg12, seg3])

conv_final = Conv2D(4, 1, activation = "softmax")(added_seg123)
model = tensorflow.keras.Model(inputs=inputs, outputs=conv_final)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 256, 256, 16) 160         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 256, 256, 16) 2320        conv2d_26[0][0]                  
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 256, 256, 16) 0           conv2d_27[0][0]                  
______________________________________________________________________________________________

In [9]:
import glob
from PIL import Image
import numpy as np

In [10]:
base_path = "C:/Users/s4601130/Downloads"
x_train_files = glob.glob(base_path + "/keras_png_slices_data/keras_png_slices_train/*.png")
y_train_files =glob.glob(base_path + "/keras_png_slices_data/keras_png_slices_seg_train/*.png")
x_val_files = glob.glob(base_path + "/keras_png_slices_data/keras_png_slices_validate/*.png")
y_val_files = glob.glob(base_path + "/keras_png_slices_data/keras_png_slices_seg_validate/*.png")
x_test_files =glob.glob(base_path + "/keras_png_slices_data/keras_png_slices_test/*.png")
y_test_files = glob.glob(base_path + "/keras_png_slices_data/keras_png_slices_seg_test/*.png")

In [11]:
x_train_files.sort()
y_train_files.sort()
x_val_files.sort()
y_val_files.sort()
x_test_files.sort()
y_test_files.sort()
print(x_val_files[0])
print(y_val_files[0])

C:/Users/s4601130/Downloads/keras_png_slices_data/keras_png_slices_validate\case_402_slice_0.nii.png
C:/Users/s4601130/Downloads/keras_png_slices_data/keras_png_slices_seg_validate\seg_402_slice_0.nii.png


In [12]:
x_train = []
y_train = []
x_val= []
y_val = []
x_test =[]
y_test= []
assert(len(y_train_files) == len(x_train_files))
for i in range(len(x_train_files)):
    
    x = np.array(Image.open(x_train_files[i]))
    x_train.append(x)
    x = np.array(Image.open(y_train_files[i]))
    y_train.append(x)
x_train = np.expand_dims(np.array(x_train), -1)
y_train = np.expand_dims(np.array(y_train), -1)



print(np.array(y_train).shape)

(9664, 256, 256, 1)


In [13]:
assert(len(y_test_files) == len(x_test_files))
for i in range(len(x_test_files)):
    
    x = np.array(Image.open(x_test_files[i]))
    x_test.append(x)
    x = np.array(Image.open(y_test_files[i]))
    y_test.append(x)
x_test = np.expand_dims(np.array(x_test), -1)
y_test = np.expand_dims(np.array(y_test), -1)
print(np.array(y_test).shape)

(544, 256, 256, 1)


In [14]:
assert(len(y_val_files) == len(x_val_files))
for i in range(len(x_val_files)):
    
    x = np.array(Image.open(x_val_files[i]))
    x_val.append(x)
    x = np.array(Image.open(y_val_files[i]))
    y_val.append(x)
x_val = np.expand_dims(np.array(x_val), -1)
y_val = np.expand_dims(np.array(y_val), -1)

In [15]:
y_train = y_train/85

#np.unique(y_train[0])
y_test = y_test/85

y_val = y_val/85
#np.unique(y_val[0])
np.unique(y_train[0])

array([0., 1., 2., 3.])

In [18]:
model.compile(loss='sparse_categorical_crossentropy',
optimizer=tensorflow.keras.optimizers.Adam(learning_rate=5e-4),
metrics=['accuracy'])

In [ ]:
print(y_train.shape)
print(y_val.shape)

In [19]:
moel_result = model.fit(x_train, y_train, batch_size=8, epochs=4, validation_data = (x_val, y_val))

Train on 9664 samples, validate on 1120 samples
Epoch 1/4
9664/9664 [==============================] - 92s 10ms/sample - loss: 0.1472 - accuracy: 0.9488 - val_loss: 0.0573 - val_accuracy: 0.9767
Epoch 2/4
9664/9664 [==============================] - 88s 9ms/sample - loss: 0.0500 - accuracy: 0.9800 - val_loss: 0.0417 - val_accuracy: 0.9829
Epoch 3/4
9664/9664 [==============================] - 87s 9ms/sample - loss: 0.0395 - accuracy: 0.9841 - val_loss: 0.0430 - val_accuracy: 0.9819
Epoch 4/4
9664/9664 [==============================] - 88s 9ms/sample - loss: 0.0352 - accuracy: 0.9857 - val_loss: 0.0301 - val_accuracy: 0.9878


In [20]:
y_pred = model.predict(x_test)

In [ ]:
print(y_pred.shape)
print(y_test.shape)

In [ ]:
from tensorflow. keras import backend as K
def dice_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice
dice_sim = dice_coef(y_test, y_pred)
print(dice_sim)

In [ ]:
scores = model.evaluate(x_test,y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [22]:
def dice_score(y_true, y_pred):
    y_true = tensorflow.keras.utils.to_categorical(y_true,4, "int32")

    intersection = tensorflow.reduce_sum(y_true*y_pred, axis= [1,2])

    denominator = tensorflow.reduce_sum(y_true +y_pred, axis= [1,2])
    
    dice_score= tensorflow.reduce_mean((2.0*intersection)/(denominator+ 1e-6), axis=0)
    return dice_score
res = dice_score(y_test, y_pred)
print(res)

tf.Tensor([0.99773209 0.90456766 0.93014963 0.95998859], shape=(4,), dtype=float64)
